<a href="https://colab.research.google.com/github/IdjiotSandwiches/knn-fer/blob/create-model/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dagshub --quiet
!pip install mlflow --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.2/252.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 90.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.

In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mlflow
import dagshub
import os
import pathlib
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/MyDrive/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


In [3]:
PATH = pathlib.Path('facial-emotion-recognition')
LABELS = os.listdir(PATH)

In [3]:
dagshub.init(repo_owner='IdjiotSandwiches', repo_name='knn-fer', mlflow=True)

Accessing as IdjiotSandwiches

Initialized MLflow to track repo "IdjiotSandwiches/knn-fer"

Repository IdjiotSandwiches/knn-fer initialized!

In [4]:
K_SIZE = (31,31)
SIGMA = 4
THETA_RANGE = np.arange(0, np.pi, np.pi/32)
LAMBD = 10.0
GAMMA = 0.5
PSI = 0

gabor_params = {
    'ksize': K_SIZE,
    'sigma': SIGMA,
    'lambd': LAMBD,
    'gamma': GAMMA,
    'psi': PSI
}

In [6]:
def gabor_filter(img):
  features = []
  for theta in THETA_RANGE:
    kernel = cv.getGaborKernel(**gabor_params, theta=theta)
    filter = cv.filter2D(img, cv.CV_64F, kernel)
    features.append(filter)

  return features

In [7]:
def open_dataset(dir):
  images, labels = [], []

  for i, label in enumerate(LABELS):
    path = f'{dir}/{label}'
    for j, img_path in enumerate(os.listdir(path)):
      img = cv.imread(f'{path}/{img_path}')
      img = cv.resize(img, (48,48))
      img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
      img = img / 255.0
      img = gabor_filter(img)
      images.append(img)
      labels.append(i)

  return np.array(images), np.array(labels)

In [8]:
def save_filtered_img(images, labels):
  folder_path = f'/content/drive/MyDrive/gabor-filtered-imgs'
  os.makedirs(folder_path, exist_ok=True)

  np.save(f'{folder_path}/images.npy', images)
  np.save(f'{folder_path}/labels.npy', labels)

  print('Ok!')

In [9]:
images, labels = open_dataset(PATH)
images = images.reshape(images.shape[0], -1)
save_filtered_img(images, labels)

Ok!


In [5]:
images = np.load('/content/drive/MyDrive/gabor-filtered-imgs/images.npy')
labels = np.load('/content/drive/MyDrive/gabor-filtered-imgs/labels.npy')

In [6]:
TEST_SIZE = 0.2
RANDOM_STATE = 42

split_params = {
    'test_size': TEST_SIZE,
    'random_state': RANDOM_STATE
}

X_train, X_test, y_train, y_test = train_test_split(images, labels, **split_params)

In [ ]:
N_NEIGHBORS = [3,5]
WEIGHTS = ['uniform', 'distance']
METRIC = ['minkowski', 'euclidean', 'manhattan']
ALGORITHM = ['auto', 'ball_tree', 'kd_tree', 'brute']

for n in N_NEIGHBORS:
  for w in WEIGHTS:
    for m in METRIC:
      for a in ALGORITHM:
        model_params = {
            'n_neighbors': n,
            'weights': w,
            'metric': m,
            'algorithm': a
        }

        model = KNeighborsClassifier(**model_params)
        model.fit(X_train, y_train)

        predict = model.predict(X_test)

        metrics = {
          'accuracy': accuracy_score(y_test, predict),
          'precision': precision_score(y_test, predict, average='macro'),
          'recall': recall_score(y_test, predict, average='macro'),
          'f1': f1_score(y_test, predict, average='macro')
        }

        print(f"Accuracy: {metrics['accuracy']}\nPrecision: {metrics['precision']}\nRecall: {metrics['recall']}\nF1 Score: {metrics['f1']}")

        mlflow.set_experiment("KNN_Model Theta=32")
        mlflow.set_tracking_uri("https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow")

        with mlflow.start_run(run_name=f'KNN n_neighbors={n}, weights={w}, metric={m}, algorithm={a}'):
          mlflow.log_params(gabor_params)
          mlflow.log_params(split_params)
          mlflow.log_params(model_params)
          mlflow.log_metrics(metrics)
          mlflow.sklearn.log_model(
              sk_model=model,
              artifact_path='KNN Model',
              input_example=X_train[:1]
          )

Accuracy: 0.3074766355140187
Precision: 0.3050938501361262
Recall: 0.29387967874509396
F1 Score: 0.2783373964588805


2024/11/30 10:59:14 INFO mlflow.tracking.fluent: Experiment with name 'KNN_Model Theta=32' does not exist. Creating a new experiment.


🏃 View run KNN n_neighbors=3, weights=uniform, metric=minkowski, algorithm=auto at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4/runs/b02f353aaeb3491cb74358282af9a0b4
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4
Accuracy: 0.308411214953271
Precision: 0.30554388973180335
Recall: 0.2947200148795478
F1 Score: 0.27900485072077663


🏃 View run KNN n_neighbors=3, weights=uniform, metric=minkowski, algorithm=ball_tree at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4/runs/923932c2679642a08150dd7eb2980d57
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4
Accuracy: 0.308411214953271
Precision: 0.30564408809920407
Recall: 0.2947200148795478
F1 Score: 0.2790072898045715
🏃 View run KNN n_neighbors=3, weights=uniform, metric=minkowski, algorithm=kd_tree at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4/runs/aa15c822146048458410e873536155a7
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4
Accuracy: 0.3074766355140187
Precision: 0.3050938501361262
Recall: 0.29387967874509396
F1 Score: 0.2783373964588805
🏃 View run KNN n_neighbors=3, weights=uniform, metric=minkowski, algorithm=brute at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4/runs/df424c2fee9e46bd9b4a31f40bc56106
🧪 View exp

Accuracy: 0.308411214953271
Precision: 0.30554388973180335
Recall: 0.2947200148795478
F1 Score: 0.27900485072077663


🏃 View run KNN n_neighbors=3, weights=uniform, metric=euclidean, algorithm=ball_tree at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4/runs/eec782be7df14b27a88a49477edf3286
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4
Accuracy: 0.308411214953271
Precision: 0.30564408809920407
Recall: 0.2947200148795478
F1 Score: 0.2790072898045715
🏃 View run KNN n_neighbors=3, weights=uniform, metric=euclidean, algorithm=kd_tree at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4/runs/f6a2dd52f9ad4de7928c6899918adb7c
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4
Accuracy: 0.3074766355140187
Precision: 0.3050938501361262
Recall: 0.29387967874509396
F1 Score: 0.2783373964588805


🏃 View run KNN n_neighbors=3, weights=uniform, metric=euclidean, algorithm=brute at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4/runs/7da638b7f90b4ffc924ca1bba56e99b3
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4
Accuracy: 0.3233644859813084
Precision: 0.31787422780360425
Recall: 0.30837462993039805
F1 Score: 0.2955184378198163
🏃 View run KNN n_neighbors=3, weights=uniform, metric=manhattan, algorithm=auto at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4/runs/c0fe6cc1f71b4d0890cca4f4d474bb7d
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4
Accuracy: 0.3233644859813084
Precision: 0.3177394374501707
Recall: 0.30837462993039805
F1 Score: 0.2954523945289038
🏃 View run KNN n_neighbors=3, weights=uniform, metric=manhattan, algorithm=ball_tree at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4/runs/3b8d1d02eaf148a78fbb3c667b6a18ba
🧪 View expe

🏃 View run KNN n_neighbors=3, weights=distance, metric=minkowski, algorithm=ball_tree at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4/runs/f055b28fb9714f9fb135bc5babb520f8
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4
Accuracy: 0.34766355140186916
Precision: 0.3287131082820621
Recall: 0.3130277230955814
F1 Score: 0.31473980038813476
🏃 View run KNN n_neighbors=3, weights=distance, metric=minkowski, algorithm=kd_tree at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4/runs/149874b20f7a402a9ec2acd812da0472
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4
Accuracy: 0.34579439252336447
Precision: 0.3263228137046797
Recall: 0.3106175282484117
F1 Score: 0.3123337182888574
🏃 View run KNN n_neighbors=3, weights=distance, metric=minkowski, algorithm=brute at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4/runs/dfff4fde858b47119d34d5c534636329
🧪 Vie

Accuracy: 0.34766355140186916
Precision: 0.3286418668785575
Recall: 0.3130277230955814
F1 Score: 0.3147611975894672
🏃 View run KNN n_neighbors=3, weights=distance, metric=euclidean, algorithm=ball_tree at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4/runs/42485c29c4e0471089b6564a796b58d6
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4


Accuracy: 0.34766355140186916
Precision: 0.3287131082820621
Recall: 0.3130277230955814
F1 Score: 0.31473980038813476
🏃 View run KNN n_neighbors=3, weights=distance, metric=euclidean, algorithm=kd_tree at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4/runs/344be1df03114619a9efdab34917c339
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4
Accuracy: 0.34579439252336447
Precision: 0.3263228137046797
Recall: 0.3106175282484117
F1 Score: 0.3123337182888574
🏃 View run KNN n_neighbors=3, weights=distance, metric=euclidean, algorithm=brute at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4/runs/32696a22a213424999b7f229720bcf01
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4
Accuracy: 0.3663551401869159
Precision: 0.3502625307532238
Recall: 0.3333851627875379
F1 Score: 0.33528941745796753
🏃 View run KNN n_neighbors=3, weights=distance, metric=manhattan, algorithm=auto at: 

In [25]:
for p, y in zip(predict[:20], y_test[:20]):
  print(f'Predict: {LABELS[p]} | Real: {LABELS[y]}')

Predict: happy | Real: happy
Predict: sad | Real: fear
Predict: surprise | Real: surprise
Predict: angry | Real: sad
Predict: angry | Real: fear
Predict: surprise | Real: surprise
Predict: fear | Real: surprise
Predict: happy | Real: happy
Predict: fear | Real: fear
Predict: angry | Real: sad
Predict: sad | Real: sad
Predict: neutral | Real: angry
Predict: sad | Real: angry
Predict: happy | Real: angry
Predict: fear | Real: disgust
Predict: fear | Real: disgust
Predict: happy | Real: happy
Predict: happy | Real: happy
Predict: happy | Real: happy
Predict: happy | Real: happy
